In [ ]:
from cronjob import get_repositories
from crowdin import crowdin_download_missing_translations, crowdin_request, delete_code_translations, delete_translation, get_orphaned_files_as_data_frame
from crowdin_sync import get_repository_state, update_repository
from datetime import datetime
from file_manager import get_crowdin_file, get_local_file
import git
import json
import os
import pandas as pd
import re
from repository import get_subrepositories, initial_dir
from zendesk import download_zendesk_articles, update_zendesk_articles, update_zendesk_translation, zendesk_get_request

In [ ]:
repositories_df = pd.read_csv('repositories.csv', comment='#')
repositories_df

In [ ]:
def get_update_result(index):
    repository = get_repositories(False)[index]
    return repository, get_repository_state(repository)

# Update ZenDesk articles

In [ ]:
repository = get_repositories(False)[0]

In [ ]:
# from cronjob import prod_domain
# from zendesk import download_zendesk_articles
# articles = download_zendesk_articles(repository.github.git_root, prod_domain)

In [ ]:
from cronjob import prod_domain, zendesk_cronjob
zendesk_cronjob(prod_domain)

# Test repository updates

In [ ]:
repositories = get_repositories(False)[3:7]
for repository in repositories:
    assert(repository.github.origin == 'holatuwol/liferay-docs')

In [ ]:
from zendesk_docs import add_mt_disclaimers

In [ ]:
for repository in repositories:
    update_result = update_repository(repository, check_upstream=True)
    
    new_files, all_files, file_info = update_result
    
    add_mt_disclaimers(repository, file_info)

In [ ]:
from crowdin_sync import update_translations

In [ ]:
for repository in repositories:
    update_result = get_repository_state(repository)
    assert(repository.github.origin == 'holatuwol/liferay-docs')
    
    new_files, all_files, file_info = update_result
    
    update_translations(repository, all_files)

# Clear out auto translations

In [ ]:
repository, update_result = get_update_result(3)
assert(repository.github.origin == 'holatuwol/liferay-docs')
assert(repository.github.single_folder == 'en/develop/tutorials/articles')

In [ ]:
from repository import get_subrepositories

In [ ]:
for repository in get_subrepositories(repository):
    if repository.github.single_folder.find('02-from-liferay-6-to-liferay-7') != -1:
        continue
    
    if repository.github.single_folder.find('03-from-liferay-70-to-71') != -1:
        continue

    delete_translation_folder(repository)

# Test updating ignore list

In [ ]:
from file_manager import get_eligible_files

def add_ignore_files(repository, folder, language):
    os.chdir(repository.github.git_root)
    
    translated_files = get_eligible_files(repository, git.ls_files(folder), language)

    os.chdir(initial_dir)

    if language != 'en':
        source_files = [
            'en/' + file[3:] if file[0:3] == '%s/' % language else file.replace('/%s/' % language, '/en/')
                for file in translated_files
        ]
    else:
        source_files = translated_files
    
    entries = set()

    for x in pd.read_csv('%s/ignore.csv' % initial_dir).to_records(index=False):
        entries.add('%s/%s' % (x[0], x[1]))

        if x[1][-3:] == '.md':
            entries.add('%s/%s.markdown' % (x[0], x[1][:-3]))
    
    for x in source_files:
        entries.add('%s/%s' % (repository.github.upstream, x))

        if x[-3:] == '.md':
            entries.add('%s/%s.markdown' % (repository.github.upstream, x[:-3]))

    def get_ignore_entry(path):
        remote, repo_name, file = path.split('/', 2)
        return { 'repository': remote + '/' + repo_name, 'file': file }
    
    df = pd.DataFrame(sorted(
        [get_ignore_entry(x) for x in entries],
        key = lambda x: (x['repository'], x['file']
    )))
    
    df.to_csv('%s/ignore.csv' % initial_dir, index=False)

In [ ]:
repository, update_result = get_update_result(4)
assert(repository.github.origin == 'holatuwol/liferay-docs')

In [ ]:
new_files, all_files, file_info = update_result

In [ ]:
add_ignore_files(repository, 'ja/develop/tutorials/articles/01-introduction-to-liferay-development', 'ja')
add_ignore_files(repository, 'ja/develop/tutorials/articles/100-tooling', 'ja')
add_ignore_files(repository, 'ja/develop/tutorials/articles/120-customizing', 'ja')
add_ignore_files(repository, 'ja/develop/tutorials/articles/140-application-security', 'ja')
add_ignore_files(repository, 'ja/develop/tutorials/articles/150-web-services', 'ja')
add_ignore_files(repository, 'ja/develop/tutorials/articles/160-search', 'ja')
add_ignore_files(repository, 'ja/develop/tutorials/articles/185-liferay-js-apis', 'ja')
add_ignore_files(repository, 'ja/develop/tutorials/articles/210-front-end-taglibs', 'ja')
add_ignore_files(repository, 'ja/develop/tutorials/articles/240-product-navigation', 'ja')
add_ignore_files(repository, 'ja/develop/tutorials/articles/280-workflow', 'ja')
add_ignore_files(repository, 'ja/develop/tutorials/articles/300-internationalization', 'ja')
add_ignore_files(repository, 'ja/develop/tutorials/articles/310-application-display-templates', 'ja')
add_ignore_files(repository, 'ja/develop/tutorials/articles/380-troubleshooting', 'ja')

In [ ]:
repository, update_result = get_update_result(7)

# Test RST file updates

In [ ]:
repository, update_result = get_update_result(7)
assert(repository.github.origin == 'holatuwol/liferay-learn')

In [ ]:
new_files, all_files, file_info = update_result

In [ ]:
from crowdin_sync import update_sources

os.chdir(repository.github.git_root)
rst_files = [file for file in all_files if file[-4:] == '.rst']
update_sources(repository, rst_files)
os.chdir(initial_dir)

In [ ]:
os.chdir(repository.github.git_root)
for file in file_info.keys():
    if file.find(repository.crowdin.dest_folder) == 0 and file[-4:] == '.rst':
        local_file = get_local_file(repository, file)
        delete_code_translations(repository, local_file, file_info)
os.chdir(initial_dir)

# Delete repository from CrowdIn

In [ ]:
repository, update_result = get_update_result(3)
assert(repository.github.origin == 'holatuwol/liferay-docs')

In [ ]:
from repository import get_subrepositories
from crowdin import delete_translation_folder

In [ ]:
for subrepository in get_subrepositories(repository):
    if subrepository.crowdin.single_folder.find('02-from-liferay-6-to-liferay-7') != -1:
        continue
    if subrepository.crowdin.single_folder.find('03-from-liferay-70-to-71') != -1:
        continue
    
    delete_translation_folder(subrepository)

# Remove code blocks

In [ ]:
repository, update_result = get_update_result(7)
assert(repository.github.origin == 'holatuwol/liferay-learn')

In [ ]:
new_files, all_files, file_info = update_result

In [ ]:
for file in all_files:
    delete_code_translations(repository, file, file_info)

# Check orphaned files statistics

In [ ]:
repository, update_result = get_update_result(3)
assert(repository.github.origin == 'holatuwol/liferay-learn')

In [ ]:
df = get_orphaned_files_as_data_frame(repository, update_result)

In [ ]:
def get_crowdin_link(file_id):
    return '<a href="https://crowdin.com/translate/liferay-documentation/%s/en-ja">%s</a>' % (file_id, file_id)

In [ ]:
df.style.format({'id1': get_crowdin_link, 'id2': get_crowdin_link})

In [ ]:
df[df['translated1'] > 0].style.format({'id1': get_crowdin_link, 'id2': get_crowdin_link})